In [1]:
from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm
from sklearn.preprocessing import normalize

import seaborn as sns
import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
from ast import literal_eval

# For Python2 this have to be done
from __future__ import division

# Split Data

## Will do a time-ordered split later

In [7]:
def to_sparse_matrix(df, num_user, num_item, user_col, item_col, rating_col):

    dok = df[[user_col, item_col, rating_col]].copy()
    dok = dok.values
    dok = dok[dok[:, 2] > 0]
    shape = [num_user, num_item]

    return sparse.csr_matrix((dok[:, 2].astype(np.float32), (dok[:, 0], dok[:, 1])), shape=shape)

def leave_one_out_split(df, user_col, ratio, random_state=None):
    grouped = df.groupby(user_col, as_index=False)
    valid = grouped.apply(lambda x: x.sample(frac=ratio, random_state=random_state))
    train = df.loc[~df.index.isin([x[1] for x in valid.index])]
    return train, valid

def main(enable_validation = False):
    df = pd.read_csv('../../data/yelp/' + 'Data.csv')

    num_users = df['UserIndex'].nunique()
    num_items = df['ItemIndex'].nunique()

    df_train, df_test = leave_one_out_split(df, 'UserIndex', 0.2, random_state=8292)

    if enable_validation:
        df_train, df_valid = leave_one_out_split(df_train, 'UserIndex', 0.2, random_state=8292)

        df_valid.to_csv(args.data_dir + 'Valid.csv')
        R_valid = to_sparse_matrix(df_valid, num_users, num_items, 'UserIndex','ItemIndex', args.rating_col)
        sparse.save_npz(args.data_dir+'Rvalid.npz', R_valid)

    df_train.to_csv('../../data/yelp/'  + 'Train.csv')
    R_train = to_sparse_matrix(df_train, num_users, num_items, 'UserIndex', 'ItemIndex', 'Binary')
    sparse.save_npz('../../data/yelp/' + 'Rtrain.npz', R_train)

    df_test.to_csv('../../data/yelp/' + 'Test.csv')
    R_test = to_sparse_matrix(df_test, num_users, num_items, 'UserIndex', 'ItemIndex', 'Binary')
    sparse.save_npz('../../data/yelp/' + 'Rtest.npz', R_test)

In [8]:
main()

# Load Data

In [9]:
# Load Original Data
df_train = pd.read_csv('../../data/yelp/Train.csv')
# df_valid = pd.read_csv('../../data/yelp/Valid.csv')
df_test = pd.read_csv('../../data/yelp/Test.csv')
key_phrase = pd.read_csv('../../data/yelp/KeyPhrases.csv')

In [14]:
keyphrases = key_phrase['Phrases'].tolist()

In [15]:
len(keyphrases)

233

In [16]:
keyphrases

['chinese',
 'fast',
 'thai',
 'bar',
 'fry',
 'fried',
 'dessert',
 'dinner',
 'lunch',
 'soup',
 'green',
 'mexico',
 'italy',
 'mexican',
 'vietnamese',
 'buffet',
 'takeout',
 'casual',
 'pub',
 'bakery',
 'indian',
 'classic',
 'modern',
 'french',
 'italian',
 'asian',
 'birthday',
 'vegetarian',
 'downtown',
 'bbq',
 'japanese',
 'breakfast',
 'seafood',
 'brunch',
 'taco',
 'curry',
 'potato',
 'crispy',
 'shrimp',
 'bread',
 'chocolate',
 'ramen',
 'pizza',
 'beer',
 'sandwich',
 'cake',
 'sushi',
 'egg',
 'fish',
 'coffee',
 'ice',
 'burger',
 'cheese',
 'cream',
 'salad',
 'pork',
 'beef',
 'tea',
 'noodle',
 'meat',
 'rice',
 'chicken',
 'dim sum',
 'cocktail',
 'ice cream',
 'squid',
 'tempura',
 'tapioca',
 'donut',
 'olive',
 'espresso',
 'octopus',
 'croissant',
 'banana',
 'cookie',
 'honey',
 'cone',
 'scallop',
 'congee',
 'skewer',
 'miso',
 'lettuce',
 'pop',
 'strawberry',
 'apple',
 'avocado',
 'juice',
 'booth',
 'calamari',
 'kimchi',
 'patty',
 'sesame',
 'tar

In [12]:
# Load U-I Data 
rtrain = load_npz("../../data/yelp/Rtrain.npz")
# rvalid = load_npz("../../data/beer/Rvalid.npz")
# rtest = load_npz("../../data/beer/Rtest.npz")

In [13]:
rtrain

<2343x7456 sparse matrix of type '<type 'numpy.float32'>'
	with 29538 stored elements in Compressed Sparse Row format>

In [20]:
# Generate U_K and I_K
# For validation set
U_K = get_I_K(df_train, row_name = 'UserIndex', shape = (2343, 233))
I_K = get_I_K(df_train, row_name = 'ItemIndex', shape = (7456, 233))
# For test set
U_K_test = get_I_K(df_test, row_name = 'UserIndex', shape = (2343, 233))
I_K_test = get_I_K(df_test, row_name = 'ItemIndex', shape = (7456, 233))

100%|█████████████████████████████████████████████████████████████████████████| 31393/31393 [00:01<00:00, 17606.84it/s]


In [22]:
# Save
save_npz('../../data/yelp/U_K.npz',U_K)
save_npz('../../data/yelp/I_K.npz',I_K)
save_npz( '../../data/yelp/U_K_test.npz',U_K_test)
save_npz('../../data/yelp/I_K_test.npz',I_K_test)

In [41]:
# Load 
U_K = load_npz('../../data/yelp/U_K.npz')
I_K = load_npz('../../data/yelp/I_K.npz')

In [19]:
# Models
from sklearn.metrics.pairwise import cosine_similarity
def train(matrix_train):
    similarity = cosine_similarity(X=matrix_train, Y=None, dense_output=True)
    return similarity

def get_I_K(df, row_name = 'ItemIndex', shape = (3668,75)):
    rows = []
    cols = []
    vals = []
    for i in tqdm(range(df.shape[0])):
        key_vector = literal_eval(df['keyVector'][i])
        rows.extend([df[row_name][i]]*len(key_vector)) ## Item index
        cols.extend(key_vector) ## Keyword Index
#         if binary:
        vals.extend(np.array([1]*len(key_vector)))
#         else:
#             vals.extend(arr[arr.nonzero()])    
    return csr_matrix((vals, (rows, cols)), shape=shape)


def predict(matrix_train, k, similarity, item_similarity_en = False):
    """
    res = similarity * matrix_train    if item_similarity_en = False
    res = similarity * matrix_train.T  if item_similarity_en = True
    """
    prediction_scores = []
    
    if item_similarity_en:
        matrix_train = matrix_train.transpose()
        
    for user_index in tqdm(range(matrix_train.shape[0])):
        # Get user u's prediction scores to all users
        vector_u = similarity[user_index]

        # Get closest K neighbors excluding user u self
        similar_users = vector_u.argsort()[::-1][1:k+1]
        # Get neighbors similarity weights and ratings
        similar_users_weights = similarity[user_index][similar_users]
        similar_users_ratings = matrix_train[similar_users].toarray()

        prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]

        prediction_scores.append(np.sum(prediction_scores_u, axis=0))
    res = np.array(prediction_scores)
    
    if item_similarity_en:
        res = res.transpose()
    
    return res

def prediction(prediction_score, topK, matrix_Train):

    prediction = []

    for user_index in tqdm(range(matrix_Train.shape[0])):
        vector_u = prediction_score[user_index]
        vector_train = matrix_Train[user_index]
        if len(vector_train.nonzero()[0]) > 0:
            vector_predict = sub_routine(vector_u, vector_train, topK=topK)
        else:
            vector_predict = np.zeros(topK, dtype=np.float32)

        prediction.append(vector_predict)

    return np.vstack(prediction)


def sub_routine(vector_u, vector_train, topK=500):

    train_index = vector_train.nonzero()[1]

    vector_u = vector_u

    candidate_index = np.argpartition(-vector_u, topK+len(train_index))[:topK+len(train_index)]
    vector_u = candidate_index[vector_u[candidate_index].argsort()[::-1]]
    vector_u = np.delete(vector_u, np.isin(vector_u, train_index).nonzero()[0])

    return vector_u[:topK]


In [18]:
# Evluation 
def recallk(vector_true_dense, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_true_dense)

def precisionk(vector_predict, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_predict)


def average_precisionk(vector_predict, hits, **unused):
    precisions = np.cumsum(hits, dtype=np.float32)/range(1, len(vector_predict)+1)
    return np.mean(precisions)


def r_precision(vector_true_dense, vector_predict, **unused):
    vector_predict_short = vector_predict[:len(vector_true_dense)]
    hits = len(np.isin(vector_predict_short, vector_true_dense).nonzero()[0])
    return float(hits)/len(vector_true_dense)


def _dcg_support(size):
    arr = np.arange(1, size+1)+1
    return 1./np.log2(arr)


def ndcg(vector_true_dense, vector_predict, hits):
    idcg = np.sum(_dcg_support(len(vector_true_dense)))
    dcg_base = _dcg_support(len(vector_predict))
    dcg_base[np.logical_not(hits)] = 0
    dcg = np.sum(dcg_base)
    return dcg/idcg


def click(hits, **unused):
    first_hit = next((i for i, x in enumerate(hits) if x), None)
    if first_hit is None:
        return 5
    else:
        return first_hit/10


def evaluate(matrix_Predict, matrix_Test, metric_names =['R-Precision', 'NDCG', 'Precision', 'Recall', 'MAP'], atK = [5, 10, 15, 20, 50], analytical=False):
    """
    :param matrix_U: Latent representations of users, for LRecs it is RQ, for ALSs it is U
    :param matrix_V: Latent representations of items, for LRecs it is Q, for ALSs it is V
    :param matrix_Train: Rating matrix for training, features.
    :param matrix_Test: Rating matrix for evaluation, true labels.
    :param k: Top K retrieval
    :param metric_names: Evaluation metrics
    :return:
    """
    global_metrics = {
        "R-Precision": r_precision,
        "NDCG": ndcg,
        "Clicks": click
    }

    local_metrics = {
        "Precision": precisionk,
        "Recall": recallk,
        "MAP": average_precisionk
    }

    output = dict()

    num_users = matrix_Predict.shape[0]

    for k in atK:

        local_metric_names = list(set(metric_names).intersection(local_metrics.keys()))
        results = {name: [] for name in local_metric_names}
        topK_Predict = matrix_Predict[:, :k]

        for user_index in tqdm(range(topK_Predict.shape[0])):
            vector_predict = topK_Predict[user_index]
            if len(vector_predict.nonzero()[0]) > 0:
                vector_true = matrix_Test[user_index]
                vector_true_dense = vector_true.nonzero()[1]
                hits = np.isin(vector_predict, vector_true_dense)

                if vector_true_dense.size > 0:
                    for name in local_metric_names:
                        results[name].append(local_metrics[name](vector_true_dense=vector_true_dense,
                                                                 vector_predict=vector_predict,
                                                                 hits=hits))

        results_summary = dict()
        if analytical:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = results[name]
        else:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = (np.average(results[name]),
                                                              1.96*np.std(results[name])/np.sqrt(num_users))
        output.update(results_summary)

    global_metric_names = list(set(metric_names).intersection(global_metrics.keys()))
    results = {name: [] for name in global_metric_names}

    topK_Predict = matrix_Predict[:]

    for user_index in tqdm(range(topK_Predict.shape[0])):
        vector_predict = topK_Predict[user_index]

        if len(vector_predict.nonzero()[0]) > 0:
            vector_true = matrix_Test[user_index]
            vector_true_dense = vector_true.nonzero()[1]
            hits = np.isin(vector_predict, vector_true_dense)

            # if user_index == 1:
            #     import ipdb;
            #     ipdb.set_trace()

            if vector_true_dense.size > 0:
                for name in global_metric_names:
                    results[name].append(global_metrics[name](vector_true_dense=vector_true_dense,
                                                              vector_predict=vector_predict,
                                                              hits=hits))

    results_summary = dict()
    if analytical:
        for name in global_metric_names:
            results_summary[name] = results[name]
    else:
        for name in global_metric_names:
            results_summary[name] = (np.average(results[name]), 1.96*np.std(results[name])/np.sqrt(num_users))
    output.update(results_summary)

    return output

def explain_evaluate(matrix_Predict, matrix_Test, metric_names =['R-Precision', 'NDCG', 'Precision', 'Recall', 'MAP'], atK = [5, 10, 15, 20, 50], analytical=False):
    """
    :param matrix_U: Latent representations of users, for LRecs it is RQ, for ALSs it is U
    :param matrix_V: Latent representations of items, for LRecs it is Q, for ALSs it is V
    :param matrix_Train: Rating matrix for training, features.
    :param matrix_Test: Rating matrix for evaluation, true labels.
    :param k: Top K retrieval
    :param metric_names: Evaluation metrics
    :return:
    """
    global_metrics = {
        "R-Precision": r_precision,
        "NDCG": ndcg,
        "Clicks": click
    }

    local_metrics = {
        "Precision": precisionk,
        "Recall": recallk,
        "MAP": average_precisionk
    }

    output = dict()

    num_users = matrix_Predict.shape[0]

    for k in atK:

        local_metric_names = list(set(metric_names).intersection(local_metrics.keys()))
        results = {name: [] for name in local_metric_names}
        topK_Predict = matrix_Predict[:, :k]

        for user_index in tqdm(range(topK_Predict.shape[0])):
            vector_predict = topK_Predict[user_index]
            if len(vector_predict.nonzero()[0]) > 0:
#                 vector_true = matrix_Test[user_index]
                vector_true = np.ravel(matrix_Test.todense()[0])
                vector_true_dense = np.argsort(vector_true)[::-1][:k]
#                 vector_true_dense = vector_true.nonzero()[1]
                hits = np.isin(vector_predict, vector_true_dense)

                if vector_true_dense.size > 0:
                    for name in local_metric_names:
                        results[name].append(local_metrics[name](vector_true_dense=vector_true_dense,
                                                                 vector_predict=vector_predict,
                                                                 hits=hits))

        results_summary = dict()
        if analytical:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = results[name]
        else:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = (np.average(results[name]),
                                                              1.96*np.std(results[name])/np.sqrt(num_users))
        output.update(results_summary)

    global_metric_names = list(set(metric_names).intersection(global_metrics.keys()))
    results = {name: [] for name in global_metric_names}

    topK_Predict = matrix_Predict[:]

    for user_index in tqdm(range(topK_Predict.shape[0])):
        vector_predict = topK_Predict[user_index]

        if len(vector_predict.nonzero()[0]) > 0:
#             vector_true = matrix_Test[user_index]
#             vector_true_dense = vector_true.nonzero()[1]
            vector_true = np.ravel(matrix_Test.todense()[0])
            vector_true_dense = np.argsort(vector_true)[::-1][:k]
            hits = np.isin(vector_predict, vector_true_dense)

            # if user_index == 1:
            #     import ipdb;
            #     ipdb.set_trace()

            if vector_true_dense.size > 0:
                for name in global_metric_names:
                    results[name].append(global_metrics[name](vector_true_dense=vector_true_dense,
                                                              vector_predict=vector_predict,
                                                              hits=hits))

    results_summary = dict()
    if analytical:
        for name in global_metric_names:
            results_summary[name] = results[name]
    else:
        for name in global_metric_names:
            results_summary[name] = (np.average(results[name]), 1.96*np.std(results[name])/np.sqrt(num_users))
    output.update(results_summary)

    return output

# Explanation Model

In [23]:
def explain(R,W2,k, model = "Cosine_similarity", item_similarity_en = True):
    """
    k: knn's hyperparameter k
    R: Rating Matrix with size U*I
    r_ij: observed rating with user i and item j 
    s_ij: explanation vector with user i and item j 
    Z: Joint Embedding/Latent Space with size U*U, generate r_ij and s_ij
    W2: Reconstruction matrix with size U*K 
    S: Output explanation prediction matrix with size U*K (dense numpy ndarray)
    """
    Z = train(R) # Cosine similarity as default
    S = predict(W2, k, Z, item_similarity_en=item_similarity_en) 
    if normalize_en == True:       
        return normalize(S) # prediction score
    return S

def predict(matrix_train, k, similarity, item_similarity_en = False, normalize_en = False):
    """
    matrix_train: Rating Matrix with size U*I
    k: knn's hyperparameter k
    similarity: Joint Embedding/Latent Space with size U*U or I*I
    
    res = similarity * matrix_train    if item_similarity_en = False
    res = similarity * matrix_train.T  if item_similarity_en = True
    
    r_ij: observed rating with user i and item j 
    s_ij: explanation vector with user i and item j 
    """
    prediction_scores = []
    
    if item_similarity_en:
        matrix_train = matrix_train.transpose()
        
    for user_index in tqdm(range(matrix_train.shape[0])):
        # Get user u's prediction scores to all users
        vector_u = similarity[user_index]

        # Get closest K neighbors excluding user u self
        similar_users = vector_u.argsort()[::-1][1:k+1]
        # Get neighbors similarity weights and ratings
        similar_users_weights = similarity[user_index][similar_users]
        similar_users_ratings = matrix_train[similar_users].toarray()

        prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]

        prediction_scores.append(np.sum(prediction_scores_u, axis=0))
    res = np.array(prediction_scores)
    
    if item_similarity_en:
        res = res.transpose()
    if normalize_en:
        res = normalize(res)
    return res

def explain_prediction(prediction_score, topK, matrix_Train):
    """
    output prediction res of the  top K items/keyphrase/whatever
    """
    prediction = []

    for user_index in tqdm(range(matrix_Train.shape[0])):
        vector_u = prediction_score[user_index]
        vector_train = matrix_Train[user_index]
        if len(vector_train.nonzero()[0]) > 0:
            vector_predict = sub_routine_explain(vector_u, vector_train, topK=topK)
        else:
            vector_predict = np.zeros(topK, dtype=np.float32)

        prediction.append(vector_predict)
    return np.vstack(prediction)
#     return prediction

def sub_routine_explain(vector_u, vector_train, topK=30):
    """
    vector_u: predicted user vector
    vector_train: true user vector
    topK: top k items in vector
    vector_u: top k items predicted
    """
#     train_index = vector_train.nonzero()[1]
#     candidate_index = np.argpartition(-vector_u, topK+75)[:topK+75] #  10 here to make res consistent
#     candidate_index = np.argpartition(-vector_u, 74)[:topK]
#     vector_u = candidate_index[vector_u[candidate_index].argsort()[::-1]]
#     vector_u = np.delete(vector_u, np.isin(vector_u, train_index).nonzero()[0])

    candidate_index = np.argsort(vector_u)[::-1][:topK]
    return candidate_index

def predict_pilot_explanation(explanation_scores, top_keyphrase = 10):
    """
    Used for retrieve the 1st row of prediction scores, used for pilot test
    """
    explanation = []
    for explanation_score in tqdm(explanation_scores):
        explanation.append(np.argsort(explanation_score)[::-1][:top_keyphrase])
    return np.array(explanation)

In [24]:
def explain(R,W2,k):
    """
    R: Rating Matrix with size U*I
    r_ij: observed rating with user i and item j 
    s_ij: explanation vector with user i and item j 
    Z: Joint Embedding/Latent Space with size U*U, generate r_ij and s_ij
    W2: Reconstruction matrix with size U*K 
    S: Output explanation prediction matrix with size U*K
    """
    Z = train(R)
    S = predict(W2, k, Z)
    return normalize(S)

# Evaluation Model
def recall(vector_true_dense, vector_true_predict):
    """
    The fraction of relevant instances that have been retrieved over the total amount of relevant instances
    The length of vector_true_dense and vector_true_predict has to be the same
    Out put recall
    """
    hits = len(np.isin(vector_true_predict, vector_true_dense).nonzero()[0])
    return float(hits)/len(vector_true_dense)

# Total Recall
def recall_all(true_matrix, predict_matrix, topK = 20):
    res = []
    for i in tqdm(range(len(Explanation_res1))):
        true_vector = np.argsort(np.ravel(normalize(true_matrix).todense()[i]))[-topK:]
        predict_vector = np.argsort(predict_matrix[i])[-topK:]
        res.append(recall(true_vector,predict_vector))
    return sum(res)/len(res)

# Evaluation

In [25]:
# ( U_I * I_U ) * U_K
similarity = train(rtrain)
explanation_scores = predict(U_K, 100, similarity)
# explanation =  predict_explanation(explanation_scores)
explanation = explain_prediction(explanation_scores, 100, U_K)

100%|████████████████████████████████████████████████████████████████████████████| 2343/2343 [00:00<00:00, 7607.15it/s]


In [28]:
explain_evaluate(explanation, U_K_test, atK=[5,20,40]) 

100%|█████████████████████████████████████████████████████████████████████████████| 2343/2343 [00:02<00:00, 882.49it/s]


{'MAP@20': (0.45316721230242296, 0.0019350810812115379),
 'MAP@40': (0.5842206191206047, 0.002377891946403873),
 'MAP@5': (0.623118509034002, 0.0029496286180747486),
 'NDCG': (0.7268562260242788, 0.0021141846532503274),
 'Precision@20': (0.3027102005975245, 0.0018155475022447942),
 'Precision@40': (0.5250213401621853, 0.0021673342673273256),
 'Precision@5': (0.3977806231327358, 0.0016873251019215718),
 'R-Precision': (0.5250213401621853, 0.0021673342673273256),
 'Recall@20': (0.3027102005975245, 0.0018155475022447942),
 'Recall@40': (0.5250213401621853, 0.0021673342673273256),
 'Recall@5': (0.3977806231327358, 0.0016873251019215718)}

# Tuning 

TODO

# Item-based non-personalized Explanation

In [29]:
similarity = train(np.transpose(rtrain))
explanation_scores = predict(I_K, 100, similarity)
# explanation =  predict_explanation(explanation_scores)
explanation = explain_prediction(explanation_scores, 70, I_K)

100%|████████████████████████████████████████████████████████████████████████████| 7456/7456 [00:01<00:00, 7441.12it/s]


In [31]:
explain_evaluate(explanation, I_K_test, atK=[5,20,50]) 

100%|█████████████████████████████████████████████████████████████████████████████| 7456/7456 [00:22<00:00, 327.69it/s]


{'MAP@20': (0.15952096328001145, 0.0009773669807777942),
 'MAP@5': (1.2342292923752057e-05, 2.3921726473815587e-05),
 'MAP@50': (0.36389142921916484, 0.005211973150038785),
 'NDCG': (0.3588494782611002, 0.004953922879780492),
 'Precision@20': (0.18655375754251233, 0.0015577789677544275),
 'Precision@5': (2.7427317608337905e-05, 5.315939216403462e-05),
 'Precision@50': (0.3021228743828853, 0.005048103630840563),
 'R-Precision': (0.3021228743828853, 0.005048103630840563),
 'Recall@20': (0.18655375754251233, 0.0015577789677544275),
 'Recall@5': (2.7427317608337905e-05, 5.315939216403462e-05),
 'Recall@50': (0.3021228743828853, 0.005048103630840563)}

# Create Synthetic User

In [35]:
# Get top_items items associated with keyphrase_ids 
def item_associated_with_keyphrase(I_K, keyphrase_ids, top_items = 100):
    """
    I_K: Item Keyphrase Matrix
    Keyphrase_ids: top items described by keyphrase_ids
    top_n: how many top items described by the keyphrases will be output 
    output item list (unique)
    """
    res = []
    for keyphrase_id in keyphrase_ids:
        res.append(np.argsort(np.ravel(I_K.todense()[:,keyphrase_id]))[::-1][:top_items])
    return np.unique(res)

# Modify U_U latent Space from U_I
def modify_user_preference(U_I, items, user_id = 0):
    """
    TODO: Fix the function s.t. it will not modify the initial matrix
    """
    U_I[user_id,:] = 0
    for i in items:
        U_I[0,i] = 1
    return U_I

def clear_user_keyphrase(U_K, user_id = 0):
    U_K[user_id,:] = 0

def explain_synthetic_user(rtrain, U_K, I_K, keyphrases, top_keyphrase = 20, user_id = 0, k = 100, top_items = 100, **Not_used):
    """
    
    """
    items = item_associated_with_keyphrase(I_K, keyphrases, top_items = top_items) # 8 is coffee
    U_I = modify_user_preference(rtrain, items, user_id=user_id)
    modified_U_U = train(U_I)
    U_K[user_id, :] = 0
    synthetic_user_keyphrase = normalize(predict(U_K, k, modified_U_U))[user_id]
    return np.argsort(synthetic_user_keyphrase)[::-1][:top_keyphrase]

def modify_user_keyphrase(U_K, keyphrase_ids, normalization = True, keyval = 1, user_id = 0, **Not_Used):
    """
    Change all keyphrase_ids to some fixed number, all others to 0
    Return the U_K matrix with user_id row the synthetic user1
    """
    U_K[user_id,:] = 0
    for keyphrase_id in keyphrase_ids:
        U_K[user_id,keyphrase_id] = keyval
    if normalization == True:
        return normalize(U_K)
    return U_K

In [36]:
# Modify user preference matrix
items = item_associated_with_keyphrase(I_K, [0], top_items = 200) # 'chinese'
U_I = modify_user_preference(rtrain, items, user_id = 0)

D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:19: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.


In [37]:
# make synthetic user1's keyphrase preference all 0
clear_user_keyphrase(U_K, user_id = 0)

D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:25: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.


In [42]:
U_K

<2343x233 sparse matrix of type '<type 'numpy.int32'>'
	with 298309 stored elements in Compressed Sparse Row format>

In [43]:
I_K

<7456x233 sparse matrix of type '<type 'numpy.int32'>'
	with 351924 stored elements in Compressed Sparse Row format>

In [44]:
# Get latent user similarity embedding
modified_U_U = train(U_I)
# predict
explanation_scores1 = predict(U_K, 100, modified_U_U)
explanation1 =  predict_pilot_explanation(explanation_scores1, top_keyphrase = 10)

100%|███████████████████████████████████████████████████████████████████████████| 2343/2343 [00:00<00:00, 97624.32it/s]


In [45]:
explanation1[0]

array([ 50,  60, 155, 205,  57, 160, 197,  61, 164, 226], dtype=int64)

In [65]:
def rank_in_prediction(rtrain, U_K, I_K, top_items = 200, keyphrase = 0,user_i = 0):
    """
    Get the rank for user_i with keyphrase
    TODO: modify so that no need to reload U_K,I_K
    """  
    U_K = load_npz('../../data/yelp/U_K.npz')
    U_K = normalize(U_K)
    rtrain = load_npz("../../data/yelp/Rtrain.npz")
    
    # Modify user preference matrix
    items = item_associated_with_keyphrase(I_K, keyphrase, top_items = top_items) # 'raspberry'
    U_I = modify_user_preference(rtrain, items, user_id = 0)
    
    # make synthetic user1's keyphrase preference all 0
    clear_user_keyphrase(U_K, user_id = 0)
    
    # Get latent user similarity embedding
    modified_U_U = train(U_I)
    # predict
    explanation_scores1 = predict(U_K, 100, modified_U_U)
    explanation1 =  predict_pilot_explanation(explanation_scores1, top_keyphrase = 230)
    return list(explanation1[user_i]).index(keyphrase[0])
    
def evaluate_pilot_test(rtrain,U_K, I_K,keyphrase_list, top_items = 200, user_i = 0):
    # Get the average rank for user_i with keyphrase  
    res1 = 0
    for i in range(75):
        a = rank_in_prediction(rtrain, U_K, I_K, top_items = top_items, keyphrase = [i],user_i = user_i)
        print "keyphrase", keyphrase_list[i], "'s rank is ", a
        res1+= a
    return res1/75

In [66]:
evaluate_pilot_test(rtrain,U_K,I_K,keyphrases)

D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:19: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:25: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
100%|███████████████████████████████████████████████████████████████████████████| 2343/2343 [00:00<00:00, 93719.64it/s]


keyphrase chinese 's rank is  33


100%|██████████████████████████████████████████████████████████████████████████| 2343/2343 [00:00<00:00, 101869.56it/s]


keyphrase fast 's rank is  10


100%|██████████████████████████████████████████████████████████████████████████| 2343/2343 [00:00<00:00, 101869.56it/s]


...

100%|██████████████████████████████████████████████████████████████████████████| 2343/2343 [00:00<00:00, 101869.56it/s]


keyphrase banana 's rank is  126


100%|██████████████████████████████████████████████████████████████████████████| 2343/2343 [00:00<00:00, 101869.56it/s]


keyphrase cookie 's rank is  97


41.32

# Create Synthetic item

In [73]:
# Get top_users items associated with keyphrase_ids 
def users_with_keyphrase_preference(U_K, keyphrase_ids, top_users = 100, norm = True):
    """
    U_K: User Keyphrase Matrix
    Keyphrase_ids: top_users who like keyphrase_ids
    output item list (unique)
    """
    res = []
    if norm:
        U_K = normalize(U_K)
    try:
        for keyphrase_id in keyphrase_ids:
            res.append(np.argsort(np.ravel(U_K.todense()[:,keyphrase_id]))[::-1][:top_users])
    except:
        return np.argsort(np.ravel(U_K.todense()[:,keyphrase_ids]))[::-1][:top_users]
    return np.unique(res)

# Modify I_I latent Space from U_I
def modify_item_history(U_I, users, item_id = 0):
    """
    TODO: Fix the function s.t. it will not modify the initial matrix
    """
    U_I[:,item_id] = 0
    for i in users:
        U_I[i, item_id] = 1
    return normalize(U_I)

def clear_item_keyphrase(I_K, item_id = 0):
    I_K[item_id, :] = 0

def explain_synthetic_item(rtrain, U_K, I_K, keyphrases, top_keyphrase = 20, user_id = 0, k = 100, top_users = 100, **Not_used):
    """
    
    """
    items = item_associated_with_keyphrase(I_K, keyphrases, top_items = top_items) # 8 is coffee
    U_I = modify_user_preference(rtrain, items, user_id=user_id)
    modified_U_U = train(U_I)
    U_K[user_id, :] = 0
    synthetic_user_keyphrase = normalize(predict(U_K, k, modified_U_U))[user_id]
    return np.argsort(synthetic_user_keyphrase)[::-1][:top_keyphrase]

def modify_user_keyphrase(U_K, keyphrase_ids, normalization = True, keyval = 1, user_id = 0, **Not_Used):
    """
    Change all keyphrase_ids to some fixed number, all others to 0
    Return the U_K matrix with user_id row the synthetic user1
    """
    U_K[user_id,:] = 0
    for keyphrase_id in keyphrase_ids:
        U_K[user_id,keyphrase_id] = keyval
    if normalization == True:
        return normalize(U_K)
    return U_K

## Prediction Pipeline

In [67]:
U_K = load_npz('../../data/yelp/U_K.npz')
I_K = load_npz('../../data/yelp/I_K.npz')
U_K = normalize(U_K)
I_K = normalize(I_K)
rtrain = load_npz("../../data/yelp/Rtrain.npz")

In [74]:
# Modify user preference matrix
users = users_with_keyphrase_preference(U_K, 70, top_users = 100, norm = True) # 'raspberry'
U_I = modify_item_history(rtrain, users, item_id = 0)

D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:23: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.


In [75]:
# make synthetic item1's keyphrase to all 0
clear_item_keyphrase(I_K, item_id = 0)

In [76]:
# Get latent item similarity embedding
modified_I_I = train(np.transpose(U_I))
# predict
explanation_scores1 = predict(I_K, 100, modified_I_I)
explanation1 =  predict_pilot_explanation(explanation_scores1, top_keyphrase = 70)

100%|██████████████████████████████████████████████████████████████████████████| 7456/7456 [00:00<00:00, 112969.72it/s]


In [77]:
def rank_in_prediction_item(rtrain, U_K, I_K, top_users = 200, keyphrase = 70, item_i = 0):
    """
    Get the rank for user_i with keyphrase
    TODO: modify so that no need to reload U_K,I_K
    """  
    U_K = load_npz('../../data/yelp/U_K.npz')
    I_K = load_npz('../../data/yelp/I_K.npz')
    U_K = normalize(U_K)
    I_K = normalize(I_K)
    rtrain = load_npz("../../data/yelp/Rtrain.npz")
    
    # Modify user preference matrix
    users = users_with_keyphrase_preference(U_K, keyphrase, top_users = top_users, norm = True) # 'raspberry'
    U_I = modify_item_history(rtrain, users, item_id = item_i)
    
    # make synthetic item1's keyphrase to all 0
    clear_item_keyphrase(I_K, item_id = item_i)
    
    # Get latent item similarity embedding
    modified_I_I = train(np.transpose(U_I))
    # predict
    explanation_scores1 = predict(I_K, 100, modified_I_I)
    explanation1 =  predict_pilot_explanation(explanation_scores1, top_keyphrase = 230)
    return list(explanation1[item_i]).index(keyphrase)

    
def evaluate_pilot_test_item(rtrain,U_K, I_K,keyphrase_list, top_users = 200, item_i = 0):
    # Get the average rank for item_i with keyphrase  
    res1 = 0
    for i in range(75):
        a = rank_in_prediction_item(rtrain, U_K, I_K, top_users = top_users, keyphrase = i, item_i = item_i)
        print "keyphrase", keyphrase_list[i], "'s rank is ", a
        res1+= a
    return res1/75

In [78]:
rank_in_prediction_item(rtrain, U_K, I_K, top_users = 200, keyphrase = 10, item_i = 0)

D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:23: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:29: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
100%|██████████████████████████████████████████████████████████████████████████| 7456/7456 [00:00<00:00, 112969.72it/s]


32

In [ ]:
evaluate_pilot_test_item(rtrain,U_K, I_K,keyphrases, top_users = 200, item_i = 0)